In [66]:
import requests
import json
import pandas as pd

# Racial Population Data Querying, Cleaning, and Formatting

api_key = 'd80bd509a22b6f1f21706609ad0b09afe39b2759'
api_key_end = f'&key={api_key}'

population_scales = {"State":"in=state:","Counties":"for=county:*","CongressionalDistricts":"for=congressional%20district:*"}
population_scales_savedas = {"State":"estimated_population","Counties":"counties_estimated_populations","CongressionalDistricts":"cd_estimated_total_populations"}
population_scale_keys = {"State":"","Counties":"counties","CongressionalDistricts":"cd"}

states = [24, 45]
stateNames = {24:"Maryland",45:"South Carolina"}
stateNamesLower = {24:"maryland",45:"south_carolina"}

state_race_identifiers = {"White":"S2101_C01_014E", 
                       "BlackAfricanAmerican":"S2101_C01_015E",
                       "AmericanIndianAlaskaNative":"S2101_C01_016E",
                       "Asian":"S2101_C01_017E",
                       "NativeHawaiianOtherPacificIslander":"S2101_C01_018E",
                       "HispanicOrLatino":"S2101_C01_021E",
                       "Other":"S2101_C01_019E"}
urls = []
for state in states:
  for population_scale in population_scales:
    for race_key in state_race_identifiers:
      if population_scale == "State":
        url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{state_race_identifiers[race_key]}&for=state:{state}{api_key_end}'
      elif population_scale == "Counties":
        url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{state_race_identifiers[race_key]}&{population_scales["State"]}{state}&{population_scales["Counties"]}{api_key_end}'
      elif population_scale == "CongressionalDistricts":
        url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{state_race_identifiers[race_key]}&{population_scales["State"]}{state}&{population_scales["CongressionalDistricts"]}{api_key_end}'
      # url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{state_race_identifiers[race_key]}&in=state:{state}&for=county:*&key={api_key}'
      resp = requests.get(url)
      if not resp:
        print(f'No response for {race_key} in {state}')
      if resp.status_code == 200:
        df = pd.DataFrame(resp.json())
        df = df.drop([0])

        if population_scale == "State":
          df.insert(1, "Year", [2022] * len(df), False)
          df = df.rename(columns = {0:"State"})
          df = df.rename(columns = {2:"State FIPS Code"})
        if population_scale == "Counties":
          df.insert(1, "Year", [2022] * len(df), False)
          df = df.rename(columns = {0:"County Name"})
          df['County Name']  = df['County Name'].str.replace(f', {stateNames[state]}', '')
          df = df.rename(columns = {1:"Estimated Population"})
          df = df.rename(columns = {2:"State FIPS Code"})
          df = df.rename(columns = {3:"County Number"})
        if population_scale == "CongressionalDistricts":
          df = df.rename(columns = {0:"District Name"})
          df['District Name']  = df['District Name'].str.replace(f' (118th Congress), {stateNames[state]}', '')
          df.insert(1, "Congress", [118] * len(df), False)
          df.insert(1, "Year", [2022] * len(df), False)
          df = df.rename(columns = {2:"State FIPS Code"})
          df = df.rename(columns = {3:"District Number"})
        df = df.rename(columns = {1:"Estimated Population"})
        df.to_json(f'output/{stateNamesLower[state]}_{population_scale_keys[population_scale]}_estimated_populations_{race_key}.json', orient="records", indent=4)
        print(df)
      else:
        print(f'Response code {resp.status_code} for {race_key} in {state}')

      State  Year Estimated Population State FIPS Code
1  Maryland  2022              2544431              24
      State  Year Estimated Population State FIPS Code
1  Maryland  2022              1423406              24
      State  Year Estimated Population State FIPS Code
1  Maryland  2022                14217              24
      State  Year Estimated Population State FIPS Code
1  Maryland  2022               318536              24
      State  Year Estimated Population State FIPS Code
1  Maryland  2022                 2282              24
      State  Year Estimated Population State FIPS Code
1  Maryland  2022               442573              24
      State  Year Estimated Population State FIPS Code
1  Maryland  2022               234995              24
               County Name  Year Estimated Population State FIPS Code  \
1          Allegany County  2022                49290              24   
2      Anne Arundel County  2022               309678              24   
3         B

In [ ]:
import requests
import json
import pandas as pd

# Pure Population Data Querying, Cleaning and Formatting

api_key = 'd80bd509a22b6f1f21706609ad0b09afe39b2759'
api_key_end = f'&key={api_key}'

total_population_identifier = "S0101_C01_001E"
population_scales = {"State":"in=state:","Counties":"for=county:*","CongressionalDistricts":"for=congressional%20district:*"}
population_scales_savedas = {"State":"estimated_population","Counties":"counties_estimated_populations","CongressionalDistricts":"cd_estimated_total_populations"}

states = [24, 45]
stateNames = {24:"Maryland",45:"South Carolina"}
stateNamesLower = {24:"maryland",45:"south_carolina"}

urls = []
for state in states:
  for population_scale in population_scales:
    # print(f'Getting data for {state},{population_scale}')
    # &in=state:{state}&for=congressional%20district:*&key={api_key}
    if population_scale == "State":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{total_population_identifier}&for=state:{state}{api_key_end}'
    elif population_scale == "Counties":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{total_population_identifier}&{population_scales["State"]}{state}&{population_scales["Counties"]}{api_key_end}'
    elif population_scale == "CongressionalDistricts":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{total_population_identifier}&{population_scales["State"]}{state}&{population_scales["CongressionalDistricts"]}{api_key_end}'
    resp = requests.get(url)
    if not resp:
      print(f'No response for {population_scale} in {state}')
    if resp.status_code == 200:
      df = pd.DataFrame(resp.json())
      df = df.drop([0])
      if population_scale == "State":
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {0:"State"})
        df = df.rename(columns = {2:"State FIPS Code"})
      if population_scale == "Counties":
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {0:"County Name"})
        df['County Name']  = df['County Name'].str.replace(f', {stateNames[state]}', '')
        df = df.rename(columns = {2:"State FIPS Code"})
        df = df.rename(columns = {3:"County Number"})
      if population_scale == "CongressionalDistricts":
        df = df.rename(columns = {0:"District Name"})
        df['District Name']  = df['District Name'].str.replace(f' (118th Congress), {stateNames[state]}', '')
        df.insert(1, "Congress", [118] * len(df), False)
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {2:"State"})
        df = df.rename(columns = {3:"District Number"})
      df = df.rename(columns = {1:"Estimated Population"})
      print(df)
      df.to_json(f'output/{stateNamesLower[state]}_{population_scales_savedas[population_scale]}.json', orient="records", indent=4)
    else:
      print(f'Response code {resp.status_code} for {population_scale} in {state}')

In [ ]:
import requests
import json
import pandas as pd

# Median Income Data for last 12 months Querying, Cleaning and Formatting

api_key = 'd80bd509a22b6f1f21706609ad0b09afe39b2759'
api_key_end = f'&key={api_key}'

median_income_identifier_over_18 = "S2101_C05_023E"
population_scales = {"State":"in=state:","Counties":"for=county:*","CongressionalDistricts":"for=congressional%20district:*"}
population_scales_savedas = {"State":"estimated_median_income","Counties":"counties_estimated_median_income","CongressionalDistricts":"cd_estimated_median_income"}

states = [24, 45]
stateNames = {24:"Maryland",45:"South Carolina"}
stateNamesLower = {24:"maryland",45:"south_carolina"}

urls = []
for state in states:
  for population_scale in population_scales:
    # print(f'Getting data for {state},{population_scale}')
    # &in=state:{state}&for=congressional%20district:*&key={api_key}
    if population_scale == "State":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{median_income_identifier_over_18}&for=state:{state}{api_key_end}'
    elif population_scale == "Counties":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{median_income_identifier_over_18}&{population_scales["State"]}{state}&{population_scales["Counties"]}{api_key_end}'
    elif population_scale == "CongressionalDistricts":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{median_income_identifier_over_18}&{population_scales["State"]}{state}&{population_scales["CongressionalDistricts"]}{api_key_end}'
    resp = requests.get(url)
    if not resp:
      print(f'No response for {population_scale} in {state}')
    if resp.status_code == 200:
      df = pd.DataFrame(resp.json())
      df = df.drop([0])
      if population_scale == "State":
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {0:"State"})
        df = df.rename(columns = {2:"State FIPS Code"})
      if population_scale == "Counties":
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {0:"County Name"})
        df['County Name']  = df['County Name'].str.replace(f', {stateNames[state]}', '')
        df = df.rename(columns = {2:"State FIPS Code"})
        df = df.rename(columns = {3:"County Number"})
      if population_scale == "CongressionalDistricts":
        df = df.rename(columns = {0:"District Name"})
        df['District Name']  = df['District Name'].str.replace(f' (118th Congress), {stateNames[state]}', '')
        df.insert(1, "Congress", [118] * len(df), False)
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {2:"State"})
        df = df.rename(columns = {3:"District Number"})
      df = df.rename(columns = {1:"Median Income"})
      print(df)
      df.to_json(f'output/{stateNamesLower[state]}_{population_scales_savedas[population_scale]}.json', orient="records", indent=4)
    else:
      print(f'Response code {resp.status_code} for {population_scale} in {state}')

In [65]:
import requests
import json
import pandas as pd

# Poverty or Below for last 12 months Querying, Cleaning and Formatting

api_key = 'd80bd509a22b6f1f21706609ad0b09afe39b2759'
api_key_end = f'&key={api_key}'

poverty_identifier_over_18 = "S2101_C05_036E"
population_scales = {"State":"in=state:","Counties":"for=county:*","CongressionalDistricts":"for=congressional%20district:*"}
population_scales_savedas = {"State":"estimated_poverty_population_over_18","Counties":"counties_estimated_poverty_population_over_18","CongressionalDistricts":"cd_estimated_poverty_population_over_18"}

states = [24, 45]
stateNames = {24:"Maryland",45:"South Carolina"}
stateNamesLower = {24:"maryland",45:"south_carolina"}

urls = []
for state in states:
  for population_scale in population_scales:
    # print(f'Getting data for {state},{population_scale}')
    # &in=state:{state}&for=congressional%20district:*&key={api_key}
    if population_scale == "State":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{poverty_identifier_over_18}&for=state:{state}{api_key_end}'
    elif population_scale == "Counties":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{poverty_identifier_over_18}&{population_scales["State"]}{state}&{population_scales["Counties"]}{api_key_end}'
    elif population_scale == "CongressionalDistricts":
      url = f'https://api.census.gov/data/2022/acs/acs5/subject?get=NAME,{poverty_identifier_over_18}&{population_scales["State"]}{state}&{population_scales["CongressionalDistricts"]}{api_key_end}'
    resp = requests.get(url)
    if not resp:
      print(f'No response for {population_scale} in {state}')
    if resp.status_code == 200:
      df = pd.DataFrame(resp.json())
      df = df.drop([0])
      if population_scale == "State":
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {0:"State"})
        df = df.rename(columns = {2:"State FIPS Code"})
      if population_scale == "Counties":
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {0:"County Name"})
        df['County Name']  = df['County Name'].str.replace(f', {stateNames[state]}', '')
        df = df.rename(columns = {2:"State FIPS Code"})
        df = df.rename(columns = {3:"County Number"})
      if population_scale == "CongressionalDistricts":
        df = df.rename(columns = {0:"District Name"})
        df['District Name']  = df['District Name'].str.replace(f' (118th Congress), {stateNames[state]}', '')
        df.insert(1, "Congress", [118] * len(df), False)
        df.insert(1, "Year", [2022] * len(df), False)
        df = df.rename(columns = {2:"State"})
        df = df.rename(columns = {3:"District Number"})
      df = df.rename(columns = {1:"Population in Poverty"})
      print(df)
      df.to_json(f'output/{stateNamesLower[state]}_{population_scales_savedas[population_scale]}.json', orient="records", indent=4)
    else:
      print(f'Response code {resp.status_code} for {population_scale} in {state}')

      State  Year Population in Poverty State FIPS Code
1  Maryland  2022                381719              24
               County Name  Year Population in Poverty State FIPS Code  \
1          Allegany County  2022                  7249              24   
2      Anne Arundel County  2022                 22557              24   
3         Baltimore County  2022                 56155              24   
4           Calvert County  2022                  2780              24   
5          Caroline County  2022                  2530              24   
6           Carroll County  2022                  6453              24   
7             Cecil County  2022                  6871              24   
8           Charles County  2022                  6298              24   
9        Dorchester County  2022                  3067              24   
10        Frederick County  2022                 11451              24   
11          Garrett County  2022                  2274              24   
